# TO DO: get and save JSON data from fullurls list (below)

## Generate list of unique city, state combos with expected price data

In [29]:
import pandas as pd
import numpy as np
import requests
import pickle
import progressbar

In [2]:
pd.set_option("display.max_columns",None)
cars = pd.read_pickle('../cars.pkl')
display(cars)

,post_date,lastpull_ts,link,price,year,make,model,body,mileage,title_type,city,state,seller,trim,ext_color,int_color,transmission,liters,cylinders,fuel_type,n_doors,ext_condition,int_condition,drive_type,VIN,n_pics,expected_price,zip_code
0,2020-04-05,1586150574,https://cars.ksl.com/listing/6352088,1200,1968,Oldsmobile,Toronado,Coupe,99999,Clean Title,Nibley,UT,Owner,NaN,gold,black,Automatic,NaN,8.0,Gasoline,2.0,Fair,Fair,FWD,XXXXXXXXX,8,NaN,84321.0
1,2020-04-05,1586150578,https://cars.ksl.com/listing/6320528,14823,2019,Chevrolet,Spark,Hatchback,7609,NaN,Ogden,UT,Dealer,LT,black,jet black/dark anderson silver metallic,Automatic,NaN,4.0,Gasoline,4.0,NaN,NaN,FWD,KL8CD6SAXKC805721,26,10220.0,84201.0
2,2020-04-05,1586150578,https://cars.ksl.com/listing/6328730,21997,2018,Hyundai,Tucson,Sport Utility,15311,NaN,Lindon,UT,Dealer,SEL Plus,silver,black,Automatic,NaN,4.0,Gasoline,4.0,NaN,NaN,AWD,KM8J3CA44JU813314,0,16554.0,84042.0
3,2020-04-05,1586150579,https://cars.ksl.com/listing/6293848,8500,1993,Ford,Bronco,NaN,115871,NaN,North Salt Lake,UT,Dealer,XLT,white,gray,Automatic,NaN,8.0,Gasoline,NaN,NaN,NaN,4-Wheel Drive,1FMEU15H0PLA54840,20,NaN,84054.0
4,2020-04-05,1586150579,https://cars.ksl.com/listing/6082281,12990,2013,Ram,2500,Truck,107111,Clean Title,Salt Lake City,UT,Dealer,Tradesman,white,black,Automatic,NaN,8.0,Gasoline,4.0,Excellent,Excellent,4-Wheel Drive,3C6TR5HT6DG587537,11,NaN,84101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53892,2020-03-20,1584833057,https://cars.ksl.com/listing/6269151,20148,2019,Ford,Fusion Hybrid,Sedan,20303,NaN,Price,UT,Dealer,SE,silver,NaN,NaN,NaN,4.0,Hybrid,NaN,NaN,NaN,FWD,3FA6P0LU0KR206951,0,15072.0,84501.0
53893,2020-03-20,1584833058,https://cars.ksl.com/listing/5968149,15995,2009,Jeep,Grand Cherokee,Sport Utility,122000,NaN,NaN,NaN,Dealer,SRT-8,red,charcoal gray,Automatic,NaN,8.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1J8HR78W89C505718,19,NaN,NaN
53894,2020-03-20,1584833058,https://cars.ksl.com/listing/6287241,52900,2020,Acura,MDX,Sport Utility,2080,NaN,Salt Lake City,UT,Dealer,with Advance/Entertainment Package,white,black,Automatic,NaN,6.0,Gasoline,NaN,NaN,NaN,AWD,5J8YD4H99LL011505,50,37806.0,84101.0
53895,2020-03-20,1584833059,https://cars.ksl.com/listing/6269149,22325,2019,Hyundai,Santa Fe,Sport Utility,17195,NaN,Price,UT,Dealer,SE,quartz white,espresso/gray,Automatic,NaN,4.0,Gasoline,NaN,NaN,NaN,AWD,5NMS2CAD0KH106182,21,20292.0,84501.0


In [4]:
# create a city, state column
cars['citystate'] = cars[['city', 'state']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# only get non-null expected price rows
good_cars = cars[cars.expected_price.notnull()]

In [5]:
# Generate and display list city, state combo that have cars with expected price data

citystate = good_cars['citystate'].unique()
print(f'The number of unique city, state combos is: {len(citystate)}')
print()
for combo in citystate:
    print(combo)

The number of unique city, state combos is: 282

Ogden UT
Lindon UT
North Salt Lake UT
Springville UT
Preston ID
Murray UT
Sandy UT
Twin Falls ID
Pocatello ID
West Haven UT
Woods Cross UT
St. George UT
Salt Lake City UT
Rexburg ID
Smithfield UT
Santaquin UT
Midvale UT
North Logan UT
American Fork UT
Vernal UT
Gardena CA
Orem UT
Clearfield UT
Clinton UT
Cedar City UT
Washington UT
Boise ID
Tremonton UT
Heber City UT
Blackfoot ID
Richfield UT
West Valley City UT
Kearns UT
South Jordan UT
Layton UT
Riverton UT
Pleasant Grove UT
Caldwell ID
Price UT
Salem UT
Kingston UT
Garland UT
Vineyard UT
Rock Springs WY
Elk Ridge UT
West Jordan UT
Challis ID
Hurricane UT
Las Vegas NV
Nephi UT
Rigby ID
Saratoga Springs UT
Idaho Falls ID
Payson UT
Bountiful UT
Pinedale WY
Elko NV
Delta UT
Millcreek UT
Draper UT
West Point UT
South Ogden UT
Provo UT
Spanish Fork UT
Wellington UT
Harrisville UT
Syracuse UT
Mona UT
Lehi UT
Park City UT
Roosevelt UT
Roy UT
Copperton UT
Hyrum UT
Logan UT
Cottonwood Heights U

In [6]:
# Use city state combo to get polygons from openstreemap

lefturl = 'https://nominatim.openstreetmap.org/search.php?q='
righturl = '&polygon_geojson=1&viewbox=&format=json'

# example with 1 combo
middurl = '+'.join(citystate[0].split(' '))

fullurl = lefturl + middurl + righturl
fullurl

'https://nominatim.openstreetmap.org/search.php?q=Ogden+UT&polygon_geojson=1&viewbox=&format=json'

In [7]:
# use requests to get json data

resp = requests.get(fullurl)
resp.json()

[{'place_id': 233429531,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 198889,
  'boundingbox': ['41.160109', '41.285993', '-112.0255709', '-111.920381'],
  'lat': '41.2230048',
  'lon': '-111.9738429',
  'display_name': 'Ogden, Weber County, Utah, United States of America',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.7215661835645191,
  'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png',
  'geojson': {'type': 'Polygon',
   'coordinates': [[[-112.0255709, 41.2088118],
     [-112.0228458, 41.2088038],
     [-112.0226258, 41.2087674],
     [-112.0242727, 41.2067899],
     [-112.0242244, 41.2065356],
     [-112.0236987, 41.2067294],
     [-112.0231569, 41.2058536],
     [-112.0225185, 41.2061038],
     [-112.0219231, 41.2064953],
     [-112.0214295, 41.2069311],
     [-112.0206356, 41.2074921],
     [-112.018286, 41.2074437],
     [-112.01

In [45]:
# Now generate a list of all urls with list comprehension

fullurls = [lefturl + '+'.join(middurl.split(' ')) + righturl for middurl in citystate]
fullurls = dict(zip(citystate, fullurls))
fullurls

{'Ogden UT': 'https://nominatim.openstreetmap.org/search.php?q=Ogden+UT&polygon_geojson=1&viewbox=&format=json',
 'Lindon UT': 'https://nominatim.openstreetmap.org/search.php?q=Lindon+UT&polygon_geojson=1&viewbox=&format=json',
 'North Salt Lake UT': 'https://nominatim.openstreetmap.org/search.php?q=North+Salt+Lake+UT&polygon_geojson=1&viewbox=&format=json',
 'Springville UT': 'https://nominatim.openstreetmap.org/search.php?q=Springville+UT&polygon_geojson=1&viewbox=&format=json',
 'Preston ID': 'https://nominatim.openstreetmap.org/search.php?q=Preston+ID&polygon_geojson=1&viewbox=&format=json',
 'Murray UT': 'https://nominatim.openstreetmap.org/search.php?q=Murray+UT&polygon_geojson=1&viewbox=&format=json',
 'Sandy UT': 'https://nominatim.openstreetmap.org/search.php?q=Sandy+UT&polygon_geojson=1&viewbox=&format=json',
 'Twin Falls ID': 'https://nominatim.openstreetmap.org/search.php?q=Twin+Falls+ID&polygon_geojson=1&viewbox=&format=json',
 'Pocatello ID': 'https://nominatim.openstreet

In [53]:
# get coordinates for each city and state

with progressbar.ProgressBar(maxval=len(fullurls)) as bar:
    city_state_json = {}
    i = 0
    for loc, url in fullurls.items():
        city_state_json[loc] = requests.get(url).json()
        i += 1
        bar.update(i)
    

100% (282 of 282) |######################| Elapsed Time: 0:03:05 Time:  0:03:05


In [54]:
# save location information as pickle
with open('city_coordinates.pkl', 'wb') as handle:
    pickle.dump(city_state_json,handle)

In [55]:
# Test the loading
with open('city_coordinates.pkl','rb') as handle:
    loaded_data = pickle.load(handle)
loaded_data
print(city_state_json == loaded_data)

True


# ipyleaflet

Kind of like a free version of Google Maps API

In [ ]:
import json
from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl
# if above module isn't installed, do BOTH of the following:
# pip install ipyleaflet
# jupyter nbextension enable --py --sys-prefix ipyleaflet

from ipywidgets import HTML

from polylabel import polylabel # if not installed, do pip install python-polylabel

In [ ]:
# Example from https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html

center = [38.128, 2.588]
zoom = 5

Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

In [ ]:
# now load a more local map

center = [40.7608, -111.8910] # centered on SLC, UT
zoom = 6

Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

In [ ]:
# now let's make an interactive hoverable map!

# play around with json polygon data for St. George and Hurricane, UT

# instructions for where to get geojson data: https://gis.stackexchange.com/questions/183248/getting-polygon-boundaries-of-city-in-json-from-google-maps-api
# specifically the bit about adding '&format=json' to search url

# center = [40.7608, -111.8910] # SLC, UT
# center = [39.7102, -111.8363] # Nephi, UT
center = [37.0965, -113.5684] # St. George, UT
zoom = 9
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m.add_control(FullScreenControl())

# parse St. George data
with open('polygons\st_george_ut.json') as city_file:
    city_data = json.load(city_file)
city_data_1 = city_data[-1]['geojson']
city_data_1['varinput'] = 'mystring'

# parse Hurricane data
with open('polygons\hurricane_ut.json') as city_file:
    city_data = json.load(city_file)
city_data_2 = city_data[-1]['geojson']
city_data_2['varinput'] = 'mystring2'
    
def tmpfunc(event, feature):
#     print(event)
    
    # remove old popup layer
    if isinstance(m.layers[-1], Popup):
        m.remove_layer(m.layers[-1])
    
    # get center of current polygon
    txt_loc = polylabel(feature['coordinates'])[::-1]

    # add a popup layer on hover over a city
    message = HTML()
    message.value = feature['varinput']
    popup = Popup(location=txt_loc, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

    m.add_layer(popup) # add the new layer

# create the GeoJSON layers (polygons overlaying each city)
geo_json_1 = GeoJSON(data=city_data_1, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.3})
geo_json_2 = GeoJSON(data=city_data_2, style = {'color': 'orange', 'opacity':1, 'weight':1.9, 'fillOpacity':0.3})
geo_json_1.on_hover(tmpfunc)
geo_json_2.on_hover(tmpfunc)
m.add_layer(geo_json_1)
m.add_layer(geo_json_2)

m

# Gmaps (Google Maps)

Note: to avoid the watermark, you have to sign up for a billing service for your API account as of June 2018

In [ ]:
# Example from gmaps documentation at https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

import gmaps
import gmaps.datasets

In [ ]:
earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
earthquake_df.head()

In [ ]:
locations = earthquake_df[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

In [ ]:
new_york_coordinates = (40.75, -74.00)
gmaps.figure(center=new_york_coordinates, zoom_level=12)